In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
import seaborn as sns
import pandas as pd
import glob

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))

In [ ]:
## Load everything

filename = '/home/alberto/almacen/Source_cats/LAE_10deg_z2-5/'
files = glob.glob(filename +'data*')
files.sort()
fi = []

for name in files:
    fi.append(pd.read_csv(name))

data = pd.concat(fi, axis=0, ignore_index=True)

pm_flx = data.to_numpy()[:, 1 : 60 + 1].T
pm_err = data.to_numpy()[:, 60 + 1 : 120 + 1].T

mag_noerr = flux_to_mag(pm_flx, w_central.reshape(-1, 1))
mag_noerr[np.isnan(mag_noerr)] = 99.

pm_flx += np.random.normal(size=(pm_err.shape)) * pm_err

mag = flux_to_mag(pm_flx, w_central.reshape(-1, 1))
mag[np.isnan(mag)] = 99.

files2 = []
for i in range(len(files)):
    files2.append(f'{filename}SEDs{i + 1}.csv')
    files2.sort()
fi = []
for name in files2:
    fi.append(pd.read_csv(name, header=None))

mock = {}
mock['SEDs'] = pd.concat(fi, axis=0, ignore_index=True).to_numpy()
mock['w_Arr'] = np.load(filename + 'w_Arr.npy')

EW_lya = data['EW0'].to_numpy()
zspec = data['z'].to_numpy()

del data

In [ ]:
w_lya = 1215.67 # A
N_sources = pm_flx.shape[1]
N_sources

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.hist(EW_lya, bins=np.linspace(0, 100, 50))
ew_x = np.linspace(0, 200, 10000)
ew_dist = np.exp(-ew_x / 75)
ew_norm = simpson(ew_dist, ew_x)
#plt.plot(ew_x, ew_dist / ew_norm)
ax.set_xlabel(r'Ly$\alpha$ EW$_0$')
plt.show()

fig, ax = plt.subplots(figsize=(6, 6))
ax.hist(mag_noerr[-2], bins=np.linspace(20, 28, 50))
ax.set_xlabel('rSDSS')
plt.show()

# plt.hist(zspec, bins=np.linspace(1, 8, 50))
# plt.xlabel('z spec')
# plt.show()

In [ ]:
# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(
    pm_flx, pm_err, IGM_T_correct=True, N_nb=7, only_right=False
)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(
    pm_flx, pm_err, IGM_T_correct=False, N_nb=7
)

In [ ]:
ew0lya_min = 0
ew0lya_max = 70
ew0lya_step = 8
ew0oth_min = 0
ew0oth_max = 400
ew0oth_step = 11

other_select_list = []
for ew0min in np.linspace(ew0oth_min, ew0oth_max, ew0oth_step):
    print(ew0min)
    line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other, ew0min,
        obs=True)
    other_lines = identify_lines(line_other, pm_flx, pm_err, first=False)

    other_select_list.append(other_lines)

lya_select_list = []
lya_z_nb = []
for ew0min in np.linspace(ew0lya_min, ew0lya_max, ew0lya_step):
    print(ew0min)
    line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, ew0min)
    lya_lines, lya_cont_lines = identify_lines(line, pm_flx, pm_err, first=True)
    z_nb_Arr = np.ones(N_sources) * -1 
    wh = np.where(np.array(lya_lines) != -1)
    z_nb_Arr[wh] = z_NB(np.array(lya_cont_lines)[wh])
    
    lya_select_list.append(lya_lines)
    lya_z_nb.append(z_nb_Arr)

In [ ]:
mag_cut_min = 18
mag_cut_max = 24

z_cut_min = 2.5
z_cut_max = 3.5

EW0_cut = 25

In [ ]:
mag_cut = (mag[-2] > mag_cut_min) & (mag[-2] < mag_cut_max)
target_mag = (mag_noerr[-2] > mag_cut_min) & (mag_noerr[-2] < mag_cut_max)

select_grid = np.zeros((ew0lya_step, ew0oth_step))
rightz_grid = np.zeros((ew0lya_step, ew0oth_step))

target_z = (zspec > z_cut_min) & (zspec < z_cut_max)

target = (
    target_mag & target_z & (EW_lya > EW0_cut)
)

for i in range(ew0lya_step):
    print(i)
    # Create synthetic BBs at the right and left of the line

    lya_L = np.zeros(N_sources)
    lya_R = np.zeros(N_sources)
    lya_R2 = np.zeros(N_sources)
    lya_L_err = np.zeros(N_sources) * 99
    lya_R_err = np.zeros(N_sources) * 99

    for src in range(N_sources):
        if lya_select_list[i][src] == -1:
            continue
        l = lya_select_list[i][src]
        if l > 1:
            if l > 6:
                lya_L[src] = np.average(
                    pm_flx[l - 7 : l - 1, src],
                    weights=pm_err[l - 7 : l - 1, src] ** -2
                )
                lya_L_err[src] = np.sum(pm_err[l - 7 : l - 1, src] ** -2) ** -0.5
            else:
                lya_L[src] = np.average(
                    pm_flx[:l - 1, src],
                    weights=pm_err[:l - 1, src] ** -2
                )
                lya_L_err[src] = np.sum(pm_err[l - 7 : l - 1, src] ** -2) ** -0.5

        lya_R[src] = np.average(
            pm_flx[l + 1 : l + 8, src],
            weights=pm_err[l + 1 : l + 8, src] ** -2
        )
        lya_R2[src] = np.mean(pm_flx[l + 12 : l + 12 + 7, src])
        lya_R_err[src] = np.sum(pm_err[l + 1 : l + 8, src] ** -2) ** -0.5

    for j in range(ew0oth_step):
        line_len_Arr = np.array([len(l) for l in other_select_list[j]])
        nice_lya = (
            (np.array(lya_select_list[i]) != -1) & (line_len_Arr <= 1)
            & np.invert(lya_L - lya_R > 3 * (lya_L_err ** 2 + lya_R_err ** 2) ** 0.5)
            & (lya_R / lya_R2 > 1.)
        )
        nice_z = np.abs(lya_z_nb[i] - zspec) < 0.12
        
        z_nb_cut = (lya_z_nb[i] > z_cut_min) & (lya_z_nb[i] < z_cut_max)

        selected = mag_cut & nice_lya & z_nb_cut
        select_grid[i, j] = len(np.where(selected)[0])
        rightz_grid[i, j] = len(np.where(selected & nice_z)[0])

N_target = len(np.where(target)[0])
purity = rightz_grid / select_grid
completeness = rightz_grid / N_target

In [ ]:
# np.random.seed(23)
# for src in np.random.choice(np.where((mag[-2] < 24) & (EW_lya > 25))[0], 5):
#     fig = plt.figure(figsize=(10, 6))
#     ax = plt.gca()
#     ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])
#     # ax.plot(mock['w_Arr'], mock['SEDs'][src])
#     plt.show()
#     print('rSDSS = {0:0.2f}'.format(mag[-2, src]))

In [ ]:
fig = plt.figure(figsize=(8, 8))

width = 0.5
height = 0.5
spacing = 0.06
cbar_width = 0.05

# Define axes
ax00 = fig.add_axes([0, height + 1.5 * spacing, width, height])
ax01 = fig.add_axes([width + spacing, height + 1.5 * spacing, width, height], sharey=ax00)
ax10 = fig.add_axes([0, 0, width, height], sharex=ax00)
ax11 = fig.add_axes([width + spacing, 0, width, height], sharex=ax01, sharey=ax10)
axcbar0 = fig.add_axes([2 * width + 1.5 * spacing, height + 1.5 * spacing, cbar_width, height])
axcbar1 = fig.add_axes([2 * width + 1.5 * spacing, 0, cbar_width, height])

# Plot stuff in the rectangles
vmax = np.max([np.max(rightz_grid), np.max(select_grid)])
cmap = 'Spectral'

sns.heatmap(rightz_grid, ax=ax00, vmin=0, vmax=vmax, cbar_ax=axcbar0, cmap=cmap)
sns.heatmap(select_grid, ax=ax01, vmin=0, vmax=vmax, cbar_ax=axcbar0, cmap=cmap)

sns.heatmap(purity, ax=ax10, vmin=0, vmax=1, cbar_ax=axcbar1, cmap=cmap)
sns.heatmap(completeness, ax=ax11, vmin=0, vmax=1, cbar=False, cmap=cmap)

ax00.invert_yaxis()
ax10.invert_yaxis()

# Axes ticks
xticks = range(ew0oth_step)
yticks = range(ew0lya_step)
xtick_labels = ['{0:0.0f}'.format(n) for n in np.linspace(ew0oth_min, ew0oth_max, ew0oth_step)]
ytick_labels = ['{0:0.0f}'.format(n) for n in np.linspace(ew0lya_min, ew0lya_max, ew0lya_step)]

ax00.set_yticks(yticks)
ax00.set_yticklabels(ytick_labels)
ax00.set_xticks(xticks)
ax00.set_xticklabels(xtick_labels, rotation='vertical')

ax10.set_yticks(yticks)
ax10.set_yticklabels(ytick_labels)
ax10.set_xticks(xticks)
ax10.set_xticklabels(xtick_labels, rotation='vertical')

ax11.set_xticks(xticks)
ax11.set_xticklabels(xtick_labels, rotation='vertical')
ax11.set_yticks(yticks)
ax11.set_yticklabels(ytick_labels)

ax01.set_xticks(xticks)
ax01.set_xticklabels(xtick_labels, rotation='vertical')
ax01.set_yticks(yticks)
ax01.set_yticklabels(ytick_labels)

# Axes labels
ylabel = r'Ly$\alpha$ EW$_0$ ($\AA$)'
xlabel = 'Other lines EW$_\mathrm{obs}$ ($\AA$)'
ax00.set_ylabel(ylabel, fontsize=12)
ax10.set_ylabel(ylabel, fontsize=12)
ax10.set_xlabel(xlabel, fontsize=12)
ax11.set_xlabel(xlabel, fontsize=12)

# Set titles
ax00.set_title('Selected w/ correct z', fontsize=15)
ax01.set_title('All selected', fontsize=15)
ax10.set_title('Purity', fontsize=15)
ax11.set_title('Completeness', fontsize=15)

# plt.savefig('output/puri-comp_magcut-' + str(mag_cut) + '.pdf', dpi=600,
#     bbox_inches='tight')
# plt.show()

print('N_target = {}'.format(N_target))

In [ ]:
i = 0
j = 5

# Create synthetic BBs at the right and left of the line

lya_L = np.zeros(N_sources)
lya_R = np.zeros(N_sources)
lya_R2 = np.zeros(N_sources)
lya_L_err = np.zeros(N_sources) * 99
lya_R_err = np.zeros(N_sources) * 99

for src in range(N_sources):
    if lya_select_list[i][src] == -1:
        continue
    l = lya_select_list[i][src]
    if l > 1:
        if l > 6:
            lya_L[src] = np.average(
                pm_flx[l - 7 : l - 1, src],
                weights=pm_err[l - 7 : l - 1, src] ** -2
            )
            lya_L_err[src] = np.sum(pm_err[l - 7 : l - 1, src] ** -2) ** -0.5
        else:
            lya_L[src] = np.average(
                pm_flx[:l - 1, src],
                weights=pm_err[:l - 1, src] ** -2
            )
            lya_L_err[src] = np.sum(pm_err[l - 7 : l - 1, src] ** -2) ** -0.5

    lya_R[src] = np.average(
        pm_flx[l + 1 : l + 8, src],
        weights=pm_err[l + 1 : l + 8, src] ** -2
    )
    lya_R2[src] = np.mean(pm_flx[l + 12 : l + 12 + 7, src])
    lya_R_err[src] = np.sum(pm_err[l + 1 : l + 8, src] ** -2) ** -0.5

line_len_Arr = np.array([len(l) for l in other_select_list[j]])
nice_lya = (
    (np.array(lya_select_list[i]) != -1) & (line_len_Arr <= 1)
    & np.invert(lya_L - lya_R > 3 * (lya_L_err ** 2 + lya_R_err ** 2) ** 0.5)
    & (lya_R / lya_R2 > 1.)
)
nice_z = np.abs(lya_z_nb[i] - zspec) < 0.12

z_nb_cut = (lya_z_nb[i] > z_cut_min) & (lya_z_nb[i] < z_cut_max)

selected = mag_cut & nice_lya & z_nb_cut
print(len(np.where(selected)[0]))

In [ ]:
EW_nb_Arr = np.zeros(N_sources)

for src in range(N_sources):
    l = lya_select_list[i][src]
    EW_nb_Arr[src] = nb_fwhm_Arr[l] * (pm_flx[l, src] - cont_est_lya[l, src])\
        / cont_est_lya[l, src] / (1 + np.array(lya_z_nb[i][src]))

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))

bins = np.linspace(0, 200, 30)

ax.hist(EW_nb_Arr[selected & nice_z], bins=bins, histtype='step', label='Nice z')
ax.hist(EW_nb_Arr[selected & ~nice_z], bins=bins, histtype='step', label='Bad z')

ax.set_xlim((0, 200))
ax.legend(fontsize=15)
plt.show()

In [ ]:
for k, src in enumerate(np.where(selected)[0]):
    if k == 20: break
    fig = plt.figure(figsize=(15, 6))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])
    ax.plot(mock['w_Arr'], mock['SEDs'][src])
    print(f'z_NB = {lya_z_nb[i][src]}')
    print(f'EW0_lya = {EW_lya[src]}')
    ax.axvline(w_central[lya_select_list[i][src]], label='Selected NB')
    ax.axvline(w_lya * (1 + zspec[src]), ls='--', c='red', label=r'Lya $\lambda$')
    ax.plot(w_central[1:54], cont_est_lya[1:-2, src], ls='--', c='black')

    ax.legend(fontsize=13)
    plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))
ax.hist2d(zspec[selected], lya_z_nb[i][selected],
    bins=(np.linspace(2, 4, 20), np.linspace(2, 4, 20)))
ax.set_xlabel('z spec', fontsize=15)
ax.set_ylabel('z NB', fontsize=15)
x = np.linspace(2, 4, 100)
ax.plot(x, x, c='red')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

LR_ratio = (lya_R / lya_R2)
bins = np.linspace(-2, 4, 40)

# ax.hist(LR_ratio, bins=bins, histtype='step', label='All')
ax.hist(LR_ratio[nice_lya & ~nice_z], bins=bins, histtype='step', label='Bad z')
ax.hist(LR_ratio[nice_lya & nice_z], bins=bins, histtype='step', label='Nice z')

ax.legend(fontsize=15)
plt.show()